In [1]:
import numpy as np
import pandas as pd

In [2]:
# f_name = 'evaluation_csv/network_growth validate_counter_defer_0_1000-table.csv'
# f_name = 'evaluation_csv/network_growth validate_counter_defer_500_1500-table.csv'
# f_name = 'evaluation_csv/network_growth validate_counter_defer_1000_2000-table.csv'
# f_name = 'evaluation_csv/network_growth validate_counter_defer_2000_3000-table.csv'
f_name = 'evaluation_csv/network_growth validate_counter_defer_5000_6000-table.csv'
# f_name = 'evaluation_csv/network_growth validate_counter_defer_10000_11000-table.csv'


df_raw = pd.read_csv(f_name, skiprows=6)
df = df_raw[[
    '[run number]',
    '[step]',
    'swap-to-hate',                             # Society becomes hateful when > 30% hateful nodes
    'p_defer_hateful_post',                     # Deferring probability for hateful posts

    'max-out-degree false',                     # Max degree of normals
    'max-out-degree true',                      # Max degree of haters
    'connection-density false',                 # Connection density of normals
    'connection-density true',                  # Connection density of haters
    'reciprocal-connections-percents false',    # Reciprocity of normals
    'reciprocal-connections-percents true',     # Reciprocity of haters
    'mean-follower-followee false',             # Follower/Followee of normals
    'mean-follower-followee true',              # Follower/Followee of haters
    'mean-following true false',                # Mean followers of normals
    'mean-following true true',                 # Mean followers of haters
    'mean-following false false',               # Mean followees of normals
    'mean-following false true',                # Mean followees of haters
    # 'mean-follow-prob true false',            # Mean following probabilities of hater to normal
    'mean-follow-prob true true',               # Mean following probabilities of hater to hater
    'mean-follow-prob false false',             # Mean following probabilities of normal to normal
    # 'mean-follow-prob false true',            # Mean following probabilities of normal to hater
    'connections-percents true false',          # Fraction of edges from haters to normals
    'connections-percents false true',          # Fraction of edges from normals to haters
    'percents-persons-with-degree false true 0',# Fraction of normals with out-degree 0
    'percents-persons-with-degree false true 1',# Fraction of normals with out-degree 0
    'percents-persons-with-degree true true 0', # Fraction of haters with out-degree 0
    'percents-persons-with-degree true true 1', # Fraction of haters with out-degree 0
    'percents-hateful-persons',                 # % of hateful users
    'percents-hateful-posts',                   # % of hateful posts
    'stddev-hate-score-dist',                   # StdDev of hate score distribution
    'mean-hate-score',                          # Mean of hate score distribution
    'reposts-by-haters',                        # Reposts by haters
    'reposts-by-normals',                       # Reposts by normals
    'mean-path-length false',                   # Mean post path length of normal posts
    'mean-path-length true',                    # Mean post path length of hateful posts
]]

In [3]:
df.dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 32 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   [run number]                               500 non-null    int64  
 1   [step]                                     500 non-null    int64  
 2   swap-to-hate                               500 non-null    bool   
 3   p_defer_hateful_post                       500 non-null    float64
 4   max-out-degree false                       500 non-null    int64  
 5   max-out-degree true                        500 non-null    int64  
 6   connection-density false                   500 non-null    float64
 7   connection-density true                    500 non-null    float64
 8   reciprocal-connections-percents false      500 non-null    float64
 9   reciprocal-connections-percents true       500 non-null    float64
 10  mean-follower-followee fal

In [4]:
pd.options.display.max_rows = 100
df_swap = df.copy()
df = df[df['swap-to-hate'] != True]
display(df.head(3))
print(len(df))

,[run number],[step],swap-to-hate,p_defer_hateful_post,max-out-degree false,max-out-degree true,connection-density false,connection-density true,reciprocal-connections-percents false,reciprocal-connections-percents true,...,percents-persons-with-degree true true 0,percents-persons-with-degree true true 1,percents-hateful-persons,percents-hateful-posts,stddev-hate-score-dist,mean-hate-score,reposts-by-haters,reposts-by-normals,mean-path-length false,mean-path-length true
2,2,6001,False,0.1,207,45,0.000607,0.019252,0.886219,0.685552,...,0.416667,0.229167,3.200000,0.321864,0.087285,0.406026,323.293307,13.732380,0.716173,2.812918
3,6,6001,False,0.1,205,65,0.000606,0.019885,0.888344,0.752632,...,0.341837,0.265306,3.266667,0.345120,0.103312,0.382119,323.780411,13.879782,0.674946,2.951844
4,9,6001,False,0.1,170,71,0.000607,0.017318,0.886895,0.745843,...,0.348416,0.266968,3.683333,0.354882,0.121961,0.382763,317.937785,14.021328,0.709812,2.847249


453


In [5]:
df['percents-hateful-persons'] /= 100

# Compute ratio of connection densities
df['density'] = df['connection-density true'] / df['connection-density false']

# Compute ratio of NH / HN edges
df['connections'] = df['connections-percents false true'] / df['connections-percents true false']

# Compute ratio of max out-degrees
df['ratio-max-out-degrees'] = df[df['max-out-degree true'] > 0]['max-out-degree false'] / df[df['max-out-degree true'] > 0]['max-out-degree true']

In [6]:
df_all = df.groupby(['p_defer_hateful_post']).mean()

df_plot = df_all[[
    'percents-hateful-persons',
    'percents-hateful-posts',
    'mean-hate-score',
    'stddev-hate-score-dist',

    'density',
    'ratio-max-out-degrees',
    'reciprocal-connections-percents false',
    'reciprocal-connections-percents true',

    'mean-following true false',
    'mean-following true true',
    'mean-following false false',
    'mean-following false true',
    'mean-follower-followee false',
    'mean-follower-followee true',

    'mean-path-length false',
    'mean-path-length true',
    'reposts-by-normals',
    'reposts-by-haters',
]]
df_plot.columns = [
    'Fraction of hateful persons',
    'Fraction of hateful posts',
    'Mean of hate score distribution',
    'StdDev of hate score distribution',

    'Ratio network density H / N',
    'Ratio max. followers of N / H',
    'Reciprocity of normals',
    'Reciprocity of haters',

    'Mean followers of normals',
    'Mean followers of haters',
    'Mean followees of normals',
    'Mean followees of hater',
    'Mean follower/followee of normals',
    'Mean follower/followee of haters',

    'Mean path len normals',
    'Mean path len haters',
    'Reposts of normals',
    'Reposts of haters',
]
df_plot['Swap to hate'] = df_swap.groupby(['p_defer_hateful_post'])['swap-to-hate'].mean()

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
df_plot

,Fraction of hateful persons,Fraction of hateful posts,Mean of hate score distribution,StdDev of hate score distribution,Ratio network density H / N,Ratio max. followers of N / H,Reciprocity of normals,Reciprocity of haters,Mean followers of normals,Mean followers of haters,Mean followees of normals,Mean followees of hater,Mean follower/followee of normals,Mean follower/followee of haters,Mean path len normals,Mean path len haters,Reposts of normals,Reposts of haters,Swap to hate
p_defer_hateful_post,,,,,,,,,,,,,,,,,,,
0.1,0.068135,0.428982,0.414597,0.117120,26.047763,3.174173,0.888841,0.762454,1.765240,2.344732,1.781113,2.046162,0.878599,0.782868,0.704812,2.533745,14.155852,306.170181,0.20
0.3,0.073693,0.439683,0.425438,0.120908,21.773020,3.157110,0.888308,0.757239,1.764337,2.265149,1.780376,1.984838,0.878022,0.779233,0.702810,2.250779,13.874328,252.953162,0.12
0.5,0.060105,0.383670,0.408976,0.115003,27.050670,3.107817,0.888613,0.764164,1.768785,2.336573,1.782710,2.056170,0.880095,0.777762,0.704826,2.157104,13.431148,216.249761,0.13
0.7,0.052206,0.341751,0.403373,0.116477,26.987342,3.239183,0.888799,0.745677,1.765890,2.370485,1.781512,2.047540,0.879387,0.765576,0.712265,1.988464,13.144694,162.690047,0.02
0.9,0.038588,0.239347,0.393014,0.105985,32.011243,3.970219,0.888500,0.738051,1.771127,2.425841,1.784428,2.085470,0.880215,0.756792,0.715325,1.035138,12.656863,65.935656,0.00


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=76a22323-d43c-4823-9752-036f6744cf38' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>